# Transformata Fouriera dla obrazów cyfrowych. Filtracja w dziedzinie częstotliwości.

## Cel ćwiczenia

- Zapoznanie z wykorzystaniem transformaty Fouriera w przetwarzaniu obrazów cyfrowych.
- Zapoznanie z pojęciem F-obrazu (amplitudy i fazy).
- Zapoznanie z własnościami transformaty Fouriera.
- Zapoznanie z filtracją w dziedzinie częstotliwości.

Na jednym z poprzednich ćwiczeń zetknęliśmy się z pojęciem konwolucji.
Przykładem może być filtracja dolno i górnoprzepustowa.
Operacja ta odpowiada mnożeniu w dziedzinie częstotliwości zgodnie z zależnością:

\begin{equation}
\mathcal{F}(g(x,y)*h(x,y)) = \mathcal{F}(g(x,y)) \cdot \mathcal{F}(h(x,y))
\tag{1}
\end{equation}

gdzie: $\mathcal{F}$ oznacza transformatę Fouriera, a $*$ jest splotem.

Operacja filtracji w dziedzinie częstotliwości może okazać się bardziej efektywna, jeżeli operacje $fft$ i $ifft$ (odpowiednio szybka transformata Fouriera -- *fast Fourier transform* -- oraz odwrotna szybka transformata Fouriera -- *inverse fast Fourier transform*) zajmą mniej czasu niż klasyczna konwolucja (zazwyczaj ma to miejsce dla dużego obrazu, z dużą maską).

Sama filtracja w dziedzinie częstotliwości to mnożenie punktowe całego obrazu przez jedną maskę.

W przypadku filtracji w dziedzinie częstotliwości zakłada się, że obraz "zawija się" na brzegach, co może powodować pewne artefakty (zostanie to pokazane w trakcie ćwiczenia).

W dziedzinie częstotliwości "działają" tylko filtry liniowe.
Filtry medianowe, maksymalne, minimalne itp. nie mają swoich odpowiedników.

## Dwuwymiarowa transformata Fouriera

1. Wczytaj plik "dwieFale.bmp" w skali szarości.
Jest to obraz powstały na podstawie następującej zależności:

\begin{equation}
L(m, n) = 128 + 127 \cdot \cos(\frac{2\pi m}{32}+\frac{3\pi}{4}) \cdot \cos(\frac{2\pi n}{8}-\frac{\pi}{2})
\tag{2}
\end{equation}

gdzie: $m$ i $n$ są odpowiednio numerami wierszy i kolumn.

2. Do realizacji dwuwymiarowej transformaty Fouriera służy funkcja `cv2.dft`.
Ustaw flagę `flags=cv2.DFT_COMPLEX_OUTPUT`.
Wykonaj transformatę na wczytanym obrazie.
W ten sposób uzyskuje się tzw. F-obraz.

3. Najniższe częstotliwości znajdują się w lewym-górnym rogu obrazu.
Dla celów wizualizacji (ale też przetwarzania) często wykonuje się tzw. przesunięcie F-obrazu, które powoduje, że niskie częstotliwości przesuwane są do środka obrazu.
Wykorzystaj funkcję `np.fft.fftshift`.
Jako pierwszy argument podaj wynik transformaty Fouriera.
Jako drugi argument podaj numery osi, wzdłuż których należy wykonać operację.
Pierwsza oś odnosi się do wierszy obrazu.
Druga oś odnosi się do kolumn obrazu.
Trzecia oś to część rzeczywista (`[:, :, 0]`) lub urojona (`[:, :, 1]`).
W naszym przypadku argument powinien wyglądać tak `[0,1]`.

4. Wyświetl wynik transformaty.
Na wspólnym wykresie umieść obraz oryginalny, amplitudę i fazę F-obrazu.
Amplitudę i fazę wyznacz za pomocą funkcji `cv2.cartToPolar`.
Pierwszym argumentem funkcji jest część rzeczywista wyniku, a drugim urojona.
Uwaga. W razie wątpliwości proszę sprawdzić rozmiary rezultatu transformaty Fouriera oraz przesunięcia.

5. Dla wizualizacji oblicz logarytm dziesiętny amplitudy: `ALog = np.log10(A + 1)`.
Wyświetl go zamiast amplitudy na poprzednim wykresie.

6. Wczytaj obrazy *kolo.bmp*, *kwadrat.bmp*, *kwadrat45.bmp*, *trojkat.bmp*.
Czy analizując F-obraz można coś powiedzieć o kierunku krawędzi obiektów?

7. Sprawdź (empirycznie) poprawność stwierdzenia:

`Dwuwymiarowa transformata Fouriera jest złożeniem dwóch transformat jednowymiarowych (wykonanych np. najpierw wierszowo, a później kolumnowo).` 
Jednowymiarowa transformata realizowana jest za pomocą funkcji fft (z bibloteki Numpy).

Wykonaj najpierw transformatę po wierszach: `FRow = np.fft.fft(I, axis=0)`.
Następnie po kolumnach: `FCol = np.fft.fft(FRow, axis=1)`.
Numpy zwraca wynik jako tablicę liczb zespolonych.
Część rzeczywistą można otrzymać w następujący sposób: `FCol.real`, a urojoną: `FCol.imag`.
Porównaj tak uzyskany wynik z rezultatem działania funkcji `cv2.dft`.
Można to zrobić wizualnie lub z wykorzystaniem funkcji `cv2.absdiff`.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("dwieFale.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/dwieFale.bmp --no-check-certificate
if not os.path.exists("kolo.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kolo.bmp --no-check-certificate
if not os.path.exists("kwadrat.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat.bmp --no-check-certificate
if not os.path.exists("kwadrat45.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat45.bmp --no-check-certificate
if not os.path.exists("kwadratKL.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratKL.bmp --no-check-certificate
if not os.path.exists("kwadratS.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratS.bmp --no-check-certificate
if not os.path.exists("kwadratT.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratT.bmp --no-check-certificate
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate
if not os.path.exists("trojkat.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/trojkat.bmp --no-check-certificate

I_Fale = cv2.imread('dwieFale.bmp', cv2.IMREAD_GRAYSCALE)
I_Kolo = cv2.imread('kolo.bmp', cv2.IMREAD_GRAYSCALE)
I_Kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
I_Kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)
I_Trojkat = cv2.imread('trojkat.bmp', cv2.IMREAD_GRAYSCALE)

# Function to create image based on the given equation
def create_image(rows, cols):
    L = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            L[i, j] = 128 + 127 * np.cos(2 * np.pi * i / 32 + 3 * np.pi / 4) * np.cos(2 * np.pi * j / 8 - np.pi / 2)
    return L

# Perform 2D Fourier transform for each image
images = [I_Fale, I_Kolo, I_Kwadrat, I_Kwadrat45, I_Trojkat]
titles = ['dwieFale', 'kolo', 'kwadrat', 'kwadrat45', 'trojkat']

for image, title in zip(images, titles):
    # Perform 2D Fourier transform
    F = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)

    # Shift the zero frequency component to the center
    F_shifted = np.fft.fftshift(F, axes=[0,1])

    # Calculate amplitude and phase
    magnitude, phase = cv2.cartToPolar(F_shifted[:, :, 0], F_shifted[:, :, 1])

    # Display original image, amplitude, and phase
    plt.figure(figsize=(15, 5))

    plt.subplot(131)
    plt.imshow(image, cmap='gray')
    plt.title(f'Original Image ({title})')
    plt.axis('off')

    plt.subplot(132)
    ALog = np.log10(magnitude + 1)
    plt.imshow(ALog, cmap='gray')
    plt.title(f'Logarithm of Amplitude ({title})')
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(phase, cmap='gray')
    plt.title(f'Phase ({title})')
    plt.axis('off')

    plt.show()

## Własności dwuwymiarowej transformaty Fouriera

1. Zbadaj jak zmienia się F-obraz (amplituda i faza) podczas następujących operacji: translacja, rotacja, zmiana rozmiaru, kombinacja liniowa.
Wykorzystaj stworzony wcześniej kod.<br>
Uwaga. Należy użyć przygotowanych obrazów, a nie "generować" własne.
2. Do badania translacji wykorzystaj obrazy *kwadrat.bmp* i *kwadratT.bmp*.
3. Przy badaniu rotacji wykorzystaj obrazy *kwadrat.bmp* i *kwadrat45.bmp*.
4. Przy badaniu zmiany rozmiaru wykorzystaj obrazy *kwadrat.bmp* i *kwadratS.bmp*.
5. Przy badaniu kombinacji liniowej wykorzystaj obrazy *kwadrat.bmp*, *kwadrat45.bmp* i *kwadratKL.bmp*.

In [ ]:
I_Kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
I_KwadratT = cv2.imread('kwadratT.bmp', cv2.IMREAD_GRAYSCALE)

F_Kwadrat = cv2.dft(np.float32(I_Kwadrat), flags=cv2.DFT_COMPLEX_OUTPUT)
F_KwadratT = cv2.dft(np.float32(I_KwadratT), flags=cv2.DFT_COMPLEX_OUTPUT)

F_Kwadrat_shifted = np.fft.fftshift(F_Kwadrat, axes=[0,1])
F_KwadratT_shifted = np.fft.fftshift(F_KwadratT, axes=[0,1])

magnitude_Kwadrat, phase_Kwadrat = cv2.cartToPolar(F_Kwadrat_shifted[:, :, 0], F_Kwadrat_shifted[:, :, 1])
magnitude_KwadratT, phase_KwadratT = cv2.cartToPolar(F_KwadratT_shifted[:, :, 0], F_KwadratT_shifted[:, :, 1])

plt.figure(figsize=(15, 5))

plt.subplot(231)
plt.imshow(I_Kwadrat, cmap='gray')
plt.title('Original Image (Kwadrat)')
plt.axis('off')

plt.subplot(234)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (Kwadrat)')
plt.axis('off')

plt.subplot(235)
plt.imshow(phase, cmap='gray')
plt.title('Phase (Kwadrat)')
plt.axis('off')

plt.subplot(232)
plt.imshow(I_KwadratT, cmap='gray')
plt.title('Translated Image (KwadratT)')
plt.axis('off')

plt.subplot(235)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (KwadratT)')
plt.axis('off')

plt.subplot(236)
plt.imshow(phase, cmap='gray')
plt.title('Phase (KwadratT)')
plt.axis('off')

plt.show()

I_Kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
I_Kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)

F_Kwadrat = cv2.dft(np.float32(I_Kwadrat), flags=cv2.DFT_COMPLEX_OUTPUT)
F_Kwadrat45 = cv2.dft(np.float32(I_Kwadrat45), flags=cv2.DFT_COMPLEX_OUTPUT)

F_Kwadrat_shifted = np.fft.fftshift(F_Kwadrat, axes=[0,1])
F_Kwadrat45_shifted = np.fft.fftshift(F_Kwadrat45, axes=[0,1])

magnitude_Kwadrat, phase_Kwadrat = cv2.cartToPolar(F_Kwadrat_shifted[:, :, 0], F_Kwadrat_shifted[:, :, 1])
magnitude_Kwadrat45, phase_Kwadrat45 = cv2.cartToPolar(F_Kwadrat45_shifted[:, :, 0], F_Kwadrat45_shifted[:, :, 1])

plt.figure(figsize=(15, 5))

plt.subplot(231)
plt.imshow(I_Kwadrat, cmap='gray')
plt.title('Original Image (Kwadrat)')
plt.axis('off')

plt.subplot(234)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (Kwadrat)')
plt.axis('off')

plt.subplot(235)
plt.imshow(phase, cmap='gray')
plt.title('Phase (Kwadrat)')
plt.axis('off')

plt.subplot(232)
plt.imshow(I_Kwadrat45, cmap='gray')
plt.title('Rotated Image (Kwadrat45)')
plt.axis('off')

plt.subplot(235)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (Kwadrat45)')
plt.axis('off')

plt.subplot(236)
plt.imshow(phase, cmap='gray')
plt.title('Phase (Kwadrat45)')
plt.axis('off')

plt.show()

# Read images
I_Kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
I_KwadratS = cv2.imread('kwadratS.bmp', cv2.IMREAD_GRAYSCALE)

F_Kwadrat = cv2.dft(np.float32(I_Kwadrat), flags=cv2.DFT_COMPLEX_OUTPUT)
F_KwadratS = cv2.dft(np.float32(I_KwadratS), flags=cv2.DFT_COMPLEX_OUTPUT)

F_Kwadrat_shifted = np.fft.fftshift(F_Kwadrat, axes=[0,1])
F_KwadratS_shifted = np.fft.fftshift(F_KwadratS, axes=[0,1])

magnitude_Kwadrat, phase_Kwadrat = cv2.cartToPolar(F_Kwadrat_shifted[:, :, 0], F_Kwadrat_shifted[:, :, 1])
magnitude_KwadratS, phase_KwadratS = cv2.cartToPolar(F_KwadratS_shifted[:, :, 0], F_KwadratS_shifted[:, :, 1])

plt.figure(figsize=(15, 5))

plt.subplot(231)
plt.imshow(I_Kwadrat, cmap='gray')
plt.title('Original Image (Kwadrat)')
plt.axis('off')

plt.subplot(234)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (Kwadrat)')
plt.axis('off')

plt.subplot(235)
plt.imshow(phase, cmap='gray')
plt.title('Phase (Kwadrat)')
plt.axis('off')

plt.subplot(232)
plt.imshow(I_KwadratS, cmap='gray')
plt.title('Resized Image (KwadratS)')
plt.axis('off')

plt.subplot(235)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (KwadratS)')
plt.axis('off')

plt.subplot(236)
plt.imshow(phase, cmap='gray')
plt.title('Phase (KwadratS)')
plt.axis('off')

plt.show()

I_Kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
I_Kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)
I_KwadratKL = cv2.imread('kwadratKL.bmp', cv2.IMREAD_GRAYSCALE)

F_Kwadrat = cv2.dft(np.float32(I_Kwadrat), flags=cv2.DFT_COMPLEX_OUTPUT)
F_Kwadrat45 = cv2.dft(np.float32(I_Kwadrat45), flags=cv2.DFT_COMPLEX_OUTPUT)
F_KwadratKL = cv2.dft(np.float32(I_KwadratKL), flags=cv2.DFT_COMPLEX_OUTPUT)

F_Kwadrat_shifted = np.fft.fftshift(F_Kwadrat, axes=[0,1])
F_Kwadrat45_shifted = np.fft.fftshift(F_Kwadrat45, axes=[0,1])
F_KwadratKL_shifted = np.fft.fftshift(F_KwadratKL, axes=[0,1])

magnitude_Kwadrat, phase_Kwadrat = cv2.cartToPolar(F_Kwadrat_shifted[:, :, 0], F_Kwadrat_shifted[:, :, 1])
magnitude_Kwadrat45, phase_Kwadrat45 = cv2.cartToPolar(F_Kwadrat45_shifted[:, :, 0], F_Kwadrat45_shifted[:, :, 1])
magnitude_KwadratKL, phase_KwadratKL = cv2.cartToPolar(F_KwadratKL_shifted[:, :, 0], F_KwadratKL_shifted[:, :, 1])

plt.figure(figsize=(15, 5))

plt.subplot(231)
plt.imshow(I_Kwadrat, cmap='gray')
plt.title('Original Image (Kwadrat)')
plt.axis('off')

plt.subplot(234)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (Kwadrat)')
plt.axis('off')

plt.subplot(235)
plt.imshow(phase, cmap='gray')
plt.title('Phase (Kwadrat)')
plt.axis('off')

plt.subplot(232)
plt.imshow(I_Kwadrat45, cmap='gray')
plt.title('Rotated Image (Kwadrat45)')
plt.axis('off')

plt.subplot(235)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (Kwadrat45)')
plt.axis('off')

plt.subplot(236)
plt.imshow(phase, cmap='gray')
plt.title('Phase (Kwadrat45)')
plt.axis('off')

plt.subplot(233)
plt.imshow(I_KwadratKL, cmap='gray')
plt.title('Linear Combination Image (KwadratKL)')
plt.axis('off')

plt.subplot(235)
plt.imshow(ALog, cmap='gray')
plt.title('Logarithm of Amplitude (KwadratKL)')
plt.axis('off')

plt.subplot(236)
plt.imshow(phase, cmap='gray')
plt.title('Phase (KwadratKL)')
plt.axis('off')

plt.show()

## Odwrotna dwuwymiarowa transformata Fouriera

1. Wykorzystaj stworzony wcześniej kod. Wybierz dowolny obraz np "kolo.bmp".
2. Przed realizacją odwrotnego przekszałcenia należy wykonać odwrotne przesunięcie.
Wykorzystaj funkcję `np.fft.ifftshift`.
Pierwszym argumentem jest wynik transformaty Fouriera.
Drugim argumentem są numery osi, wzdłuż których należy wykonać operację.
3. Wykonaj odwrotną transformatę Fouriera za pomocą funkcji `cv2.idft`.
Jako drugi argument przekaż następujące flagi: `flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT`.
Wynik może mieć małą część urojoną przez błędy numeryczne.
Aby pozbyć się tego efekty należy obliczyć amplitudę:
        `imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])`
Następnie wynik należy zaokrąglić (`np.round`) i zrzutować do typu `uint8`.
4. Wyświetl wynik.
Sprawdź (wizualnie i poprzez odjęcie) czy obraz oryginalny i po przekształceniach są takie same.

In [ ]:
# Read the image
I_Kolo = cv2.imread('kolo.bmp', cv2.IMREAD_GRAYSCALE)

# Perform 2D Fourier transform
F_Kolo = cv2.dft(np.float32(I_Kolo), flags=cv2.DFT_COMPLEX_OUTPUT)

# Perform inverse shift
F_Kolo_shifted = np.fft.ifftshift(F_Kolo, axes=[0,1])

# Perform inverse Fourier transform
ifft = cv2.idft(F_Kolo_shifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)

# Calculate magnitude and round the result
imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
imgIFFT_rounded = np.round(imgIFFT).astype(np.uint8)

# Display the original and reconstructed images
plt.figure(figsize=(10, 5))

plt.subplot(121)
plt.imshow(I_Kolo, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(122)
plt.imshow(imgIFFT_rounded, cmap='gray')
plt.title('Reconstructed Image')
plt.axis('off')

plt.show()

# Check if the images are the same
diff = np.abs(I_Kolo - imgIFFT_rounded)
print(f"Max absolute difference: {np.max(diff)}")

## Filtracja obrazu w dziedzinie częstotliwości

1. Wczytaj obraz "lena.bmp" w skali szarości.
Wykonaj transformatę Fouriera.
Wykorzystaj stworzony poprzednio kod.
Wyświetl obraz oryginalny, amplitudę (w skali logarytmicznej) i fazę.

2. Przeprowadź filtrację dolnoprzepustową - usuń górne częstotliwości.
Dla F-obrazu po operacji przesunięcia (`fftshift`) niskie częstotliwości leżą w jego centrum.

3. Na początku stwórz filtr "kołowy", dolnoprzepustowy.
Należy wygenerować macierze opisujące przestrzeń w dziedzinie częstotliwości.
Ich rozmiar musi być taki sam jak rozmiar przetwarzanego obrazu.

        lenaSize = I_Lena.shape
        FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
        FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
        FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
        FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)
        
Powyższy kod wygeneruje dwie znormalizowane macierze częstotliwości: *FSpaceRowsM* i *FSpaceColsM*.
Następnie należy wyznaczyć macierz zawierającą "odległość" od składowej stałej.
        `FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))`

Uwagi:
- funkcja `fftfreq` generuje wektor częstotliwości $[-0.5, 0.5]$ o określonym rozmiarze, przy czym układ wartości jest taki, że najpierw od 0 do 0.5, a potem od -0.5 do 0,
- operacja `fftshift` zmienia ten układ na $[-0.5, 0.5]$,
- mnożenie przez 2 ustala ostatecznie zakres na $[-1, 1]$,
- operacja `outer` to tzw. iloczyn zewnętrzy dwóch wektorów, w naszym przypadku powoduje, że wektor pionowy lub poziomy jest "powielany" odpowiednią liczbę razy.   
- sugeruje się, aby przyglądnąć się jak wygląda macierz `FreqR` - czy to w debugerze, czy poprzez wizualizację.

4. Teraz należy wybrać interesujący zakres.
Tu można zdefiniować typ filtru (dolno, górno, pasmowoprzepustowy).

        FilterF = FreqR <= 0.1 

Filtr należy zwizualizować:

        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()

4. Wykonaj właściwą filtrację, czyli mnożenie F-obrazu przez filtr FilterF.
Trzeba pamiętać, że F-obraz ma 2 kanały (rzeczywisty i urojony).
By mnożenie było możliwe należy więc powielić filtr również na 2 kanały.

        FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

5. Wykonaj operację odwrotnego przesunięcia i odwrotnej transformaty.
Oblicz wartość bezwzględną wyniku.
Wykorzystaj funkcję `cv2.magnitude`.
Pierwszym argumentem jest część rzeczywista.
Drugim argumentem jest część urojona.
Wynik wyświetl.

6. Poeksperymentuj z rozmiarem filtru (promieniem).
Zaimplementuj filtr górnoprzepustowy (zmiana znaku przy warunku na odległość) oraz pasmowoprzepustowy (dwa warunki na promień połączone operatorem AND '&' ).
Wykonaj co najmniej trzy filtry i wyświetl wyniki.

7. W ten sposób zaimplementowana filtracja wprowadza pewne artefakty w postaci "pierścieni" wokół krawędzi.
Zapobiec temu zjawisku można poprzez odpowiednie "modelowanie" filtra.
W tym celu wykorzystać należy okna, np. Hamminga, Hanninga, Chebysheva (znane z przetwarzania sygnałów 1D).
Zagadnienie to jest tematem zadania domowego do tego ćwiczenia.

In [ ]:
I_Lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

F_Lena = cv2.dft(np.float32(I_Lena), flags=cv2.DFT_COMPLEX_OUTPUT)

F_shift_Lena = np.fft.fftshift(F_Lena, axes=[0, 1])

magnitude_Lena, phase_Lena = cv2.cartToPolar(F_shift_Lena[:, :, 0], F_shift_Lena[:, :, 1])

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(I_Lena, cmap='gray')
axs[0].set_title('Oryginalny obraz')
axs[0].axis('off')
axs[1].imshow(np.log10(magnitude_Lena + 1), cmap='gray')
axs[1].set_title('Amplituda (skala logarytmiczna)')
axs[1].axis('off')
axs[2].imshow(phase_Lena, cmap='gray')
axs[2].set_title('Faza')
axs[2].axis('off')
plt.show()


## Implementacja wyszukiwania wzorca za pomocą FFT

1. Wczytaj w skali szarości i wyświetl obrazy *literki.bmp* i *wzorA.bmp*.

2. Wyznacz transformatę Fouriera obrazu *literki.bmp*.

3. Obróć drugi obraz o $180^\circ$.
Zastosuj funkcję `np.rot90`.
Pierwszym argumentem jest obracana macierz, a drugim liczba obrotów o $90^\circ$.

4. Należy wyznaczyć transformatę Fouriera obróconego obrazu w taki sposób, żeby miała ona taki sam rozmiar jak pierwszy obraz.
W tym celu należy zastosować *Zero Padding*.
Operacja ta polega na uzupełnieniu obrazu zerami do oczekiwanego rozmiaru.
Uzupełnij obraz zerami z **prawej** strony i z **dołu**.
W tym celu należy wykorzystać funkcję `cv2.copyMakeBorder`.
    - Pierwszym argumentem jest obraz wejściowy.
    - Drugim argumentem jest liczba wierszy u góry.
    - Trzecim argumentem jest liczba wierszy u dołu.
    - Czwartym argumentem jest liczba kolumn z lewej.
    - Piątym argumentem jest liczba kolumn z prawej.
    - Szóstym argumentem jest flaga typu wypełnienia.
    Dla stałej wartości podaj `cv2.BORDER_CONSTANT`.
    - Siódmym argumentem jest wartość pikseli w ramce.
    Przekaż `value=0`.

5. Wyznacz transformatę Fouriera obrazu stworzonego w poprzednim punkcie.

6. Wyniki obu transformat należy przekonwertować do liczb zespolonych.
Obecnie jest to dwukanałowa macierz.
Pierwszy kanał odpowiada za część rzeczywistą.
Drugi kanał odpowiada za część urojoną.
Aby to osiągnąć wystarczy wykonać działanie:

        Complex = Real + Imag * 1j

7. Przemnóż ze sobą zespolone wyniki transformat.

8. Wynik należy powrotnie przekształcić do dwukanałowej macierzy.
Aby to zrobić wykonaj operację:

        CompMat = cv2.merge([np.real(Complex), np.imag(Complex)])

9. Wykonaj odwrotną transformatę Fouriera.
Dodaj flagę `flags=cv2.DFT_COMPLEX_INPUT`.

10. Oblicz wartość bezwzględną wyniku.

11. Wykonaj morfologiczną operację **Top-Hat**, by znaleźć maksima lokalne.
Operacja ta zostanie dokładnej wyjaśniona w jednym z kolejnych ćwiczeń.
W tym celu wykorzystaj operację:

        cv2.morphologyEx(correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

12. Wyświetl obok siebie obraz wejściowy i wynik wykonanych operacji.
Czy możesz wskazać położenie wzoru na podstawie drugiego obrazu?

In [ ]:

I_Literki = cv2.imread('literki.bmp', cv2.IMREAD_GRAYSCALE)
I_WzorA = cv2.imread('wzorA.bmp', cv2.IMREAD_GRAYSCALE)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(I_Literki, cmap='gray')
plt.title('Literki')
plt.axis('off')

plt.subplot(122)
plt.imshow(I_WzorA, cmap='gray')
plt.title('WzorA')
plt.axis('off')
plt.show()

F_Literki = cv2.dft(np.float32(I_Literki), flags=cv2.DFT_COMPLEX_OUTPUT)

I_WzorA_rotated = np.rot90(I_WzorA, 2)

rows_diff = I_Literki.shape[0] - I_WzorA_rotated.shape[0]
cols_diff = I_Literki.shape[1] - I_WzorA_rotated.shape[1]
I_WzorA_padded = cv2.copyMakeBorder(I_WzorA_rotated, 0, rows_diff, 0, cols_diff, cv2.BORDER_CONSTANT, value=0)

F_WzorA = cv2.dft(np.float32(I_WzorA_padded), flags=cv2.DFT_COMPLEX_OUTPUT)

Complex_Literki = F_Literki[:, :, 0] + F_Literki[:, :, 1]*1j
Complex_WzorA = F_WzorA[:, :, 0] + F_WzorA[:, :, 1]*1j

Complex_Multiplication = Complex_Literki * Complex_WzorA

CompMat = cv2.merge([np.real(Complex_Multiplication), np.imag(Complex_Multiplication)])

correlation = cv2.idft(CompMat, flags=cv2.DFT_COMPLEX_INPUT)

magnitude_correlation = cv2.magnitude(correlation[:, :, 0], correlation[:, :, 1])

tophat = cv2.morphologyEx(magnitude_correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(I_Literki, cmap='gray')
plt.title('Literki')
plt.axis('off')

plt.subplot(122)
plt.imshow(tophat, cmap='gray')
plt.title('Correlation Result')
plt.axis('off')
plt.show()